In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
def sql_push(df,table_name):
    df.to_sql(name = table_name, con = engine, if_exists = "replace", index = False)

def download_push(ticker, tab_name):
    sdf = yf.download(ticker, start= "2015-04-01")
    sdf.reset_index(inplace=True)
    sdf = pd.DataFrame(sdf)
    sql_push(sdf, tab_name)

In [34]:
download_push(ticker="IOC.NS", tab_name="iocl")

[*********************100%***********************]  1 of 1 completed


In [ ]:
input("Run the query in 'ioc table prep.sql' file in MySQL, once done Press Enter to Continue ")

         date  close  log_close
0  2015-11-24  37.59       3.63
1  2015-11-25  37.59       3.63
2  2015-11-26  37.82       3.63
3  2015-11-27  37.68       3.63
4  2015-11-28  37.68       3.63


In [2]:
%run "data-sql-ingestion.ipynb"

         date  close  log_close
0  2015-11-24  37.59       3.63
1  2015-11-25  37.59       3.63
2  2015-11-26  37.82       3.63
3  2015-11-27  37.68       3.63
4  2015-11-28  37.68       3.63
            close  log_close
date                        
2015-11-24  37.59       3.63
2015-11-25  37.59       3.63
2015-11-26  37.82       3.63
2015-11-27  37.68       3.63
2015-11-28  37.68       3.63
